<a href="https://colab.research.google.com/github/Ayanlola2002/DATA-SCIENCE-PROJECTS/blob/master/Task5_NEEDENERGYAPI_data_hourly_id_47803.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#mounting Gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np

In [5]:
df=pd.read_csv('/content/drive/MyDrive/NEEDENERGY/data_hourly_id_47803.csv')

In [6]:
df.head()

,date,timestamp,consumption,solar,alwaysOn,gridImport,gridExport,selfConsumption,selfSufficiency,active,reactive,voltages,phaseVoltages,currentHarmonics,voltageHarmonics
0,2020-03-15 18:00:00,1584288000000,0.000,0.0,0.0,0.000,0.0,0.0,0.0,"[1.3, 78.9, -45.1, -297.3, -163.6, 321.1, 143....","[6.2, 38.9, 15.3, 534.8, 1134.3, 151.0, 1309.9...","[245.8, None, None]","[245.8, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
1,2020-03-15 19:00:00,1584291600000,913.000,0.0,0.0,913.000,0.0,0.0,0.0,"[1.2, 596.1, -509.8, 72.6, 0.0, 907.0, 2273.7,...","[6.0, 343.5, 228.7, 262.0, 0.2, 439.9, 2077.9,...","[245.6, None, None]","[245.6, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
2,2020-03-15 20:00:00,1584295200000,623.471,0.0,0.0,623.471,0.0,0.0,0.0,"[1.5, 44.8, 0.0, None, None, 387.2, 1148.0, 6....","[7.4, 10.8, 0.0, None, None, 155.4, 1374.2, 3....","[244.9, None, None]","[244.9, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
3,2020-03-15 21:00:00,1584298800000,765.984,0.0,0.0,765.984,0.0,0.0,0.0,"[1.2, 44.4, 0.0, None, None, 332.9, 1118.7, 4....","[6.0, 10.8, 0.0, None, None, 145.8, 1314.5, 2....","[244.4, None, None]","[244.4, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."
4,2020-03-15 22:00:00,1584302400000,585.041,0.0,0.0,585.041,0.0,0.0,0.0,"[1.5, 44.4, 0.0, None, None, 329.0, 1414.6, 5....","[7.5, 10.8, 0.0, None, None, 155.1, 1395.1, 3....","[244.8, None, None]","[244.8, None, None]","[[], [], [], [], [], [], [], [], [], [], [], [...","[[], [], [], [], [], [], [], [], [], [], [], [..."


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7979 entries, 0 to 7978
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              7979 non-null   object 
 1   timestamp         7979 non-null   int64  
 2   consumption       7979 non-null   float64
 3   solar             7979 non-null   float64
 4   alwaysOn          7979 non-null   float64
 5   gridImport        7979 non-null   float64
 6   gridExport        7979 non-null   float64
 7   selfConsumption   7979 non-null   float64
 8   selfSufficiency   7979 non-null   float64
 9   active            7979 non-null   object 
 10  reactive          7979 non-null   object 
 11  voltages          7979 non-null   object 
 12  phaseVoltages     7979 non-null   object 
 13  currentHarmonics  7979 non-null   object 
 14  voltageHarmonics  7979 non-null   object 
dtypes: float64(7), int64(1), object(7)
memory usage: 935.2+ KB


In [17]:
## Functions to use

def read_data(datafile, dateCol):
    ''' Read the excel file'''
    df = pd.read_excel(datafile, index_col= dateCol, parse_dates=True)
    return df
    
def info_data(datafile, info = False, head = True, shape = True):
    ''' Look at the info of the data'''
    
    if not isinstance(datafile, pd.DataFrame):
        df = read_data(datafile)
    else:
        df = datafile
    
    ## Information of data
    if info:
        df.info()
        
    print('--------------------------------------------------------------------')
    print('First rows')
    # Show first rows
    if head:
        print(df.head())
        
    print('--------------------------------------------------------------------')
    print('Dimension of the data')
    ## Dimension of data
    if shape:
        print(f'This data has {df.shape[0]} rows and {df.shape[1]} variables') 
        
        
def find_unique(df, col):
    ''' Find the unique values of a columns'''
    return df[col].nunique()

### Look at the statistical description of data
def describe(df, col = None):
    ''' Give the statistic summary'''
    
    ## Summary statistic
    if col is not None:
        stat = df[col].describe()
    else:
        stat = df.describe()

    ## Change the name of the 50% index to median
    idx = stat.index.tolist()
    idx[5] = 'median'
    stat.index = idx
    return stat

def missing_val(data):
    ''' Find the percentage of missing values
    on each column of a dataframe'''
    
    ## percentage of missing values
    n = data.isnull().sum().sort_values(ascending=False)/ len(data) * 100
    return n
  
def boxHist(df, col):
    ''' Create the boxplot and histogram'''
    ## Setup for ploting two charts one over the other
    fig, ax = plt.subplots(2, 1, figsize = (10,6))
    
    ## First a box plot
    df.dropna().boxplot(col, ax = ax[0], vert=False, return_type='dict')
    
    ## Plot the histogram
    temp = df[col].values
    ax[1].hist(temp, bins = 75)
    plt.xlabel(col)
    return [col]

def valCount(data, col, perc = False):
    ''' calculate the proportion in fraction or percentage'''
    if not perc:
      return data[col].value_counts()
    else:
      return data[col].value_counts(normalize = True).mul(100).round(1).astype(str) + '%'    
    
def make_chart_histogram(data, col = None, subplots=True, layout = None, figsize = None, sharex = False, bins = None):
    ''' Create histogram from numerical variable'''
    
    if col is not None:
        return df[col].hist(bins = bins)
    else:
        return data.plot.hist(subplots=True, layout = layout, figsize = figsize, sharex=False, bins = bins)
    
    
def plotCounts(data, col, kind = 'barh', perc = False):
    ''' Plot the proportion of a categorical variables'''
    if not perc:
        return data[col].value_counts().plot(kind = kind)
    else:
        return data[col].value_counts(normalize = True).mul(100).round(2).plot(kind = kind)

    
def convert_date(df, column_name):
    return pd.to_datetime(df[column_name])

def convertTime(df, column_name):
    return pd.to_timedelta(df[column_name])

def date_cols(data, col):
    ''' Create year, month, month name and day from a datetime column'''
    #data[col] = convert(data, col)
    
    data['Year'] = pd.DatetimeIndex(data[col]).year
    data['Month'] = pd.DatetimeIndex(data[col]).month
    data['Day'] = pd.DatetimeIndex(data[col]).day
    data['weekday'] = pd.DatetimeIndex(data[col]).weekday
    data['weekday_name'] = pd.DatetimeIndex(data[col]).day_name()
    data['Hour'] = pd.DatetimeIndex(data[col]).hour
    data['Quarter'].pd.DatetimeIndex(data[col]).Quarter

    return data    


### plots groupbyfunction
def plots(data, col1 , col2, func, title, t = 'line'):
    ''' Plotting plots from groupby'''
    
    data.groupby(col1)[col2].agg(func).plot(kind = t)
    plt.xlabel(col1)
    plt.ylabel(col2)
    plt.title(title)

def groupDat(data, col1 , col2, func):
    ''' Create groupby'''
    
    return data.groupby(col1)[col2].agg(func)     


def create_season(row):
    ''' create season'''
    if row['Month'] in (1, 2, 12):
        return "Winter"
    elif row['Month'] in (3, 4, 5):
        return "Spring"
    elif row['Month'] in (6, 7, 8):
        return "Summer"
    elif row['Month'] in (9, 10, 11):
        return "Fall"
    else:
        return "Other"

def create_day_season(row):
  if row['Hour'] in (0,1,2,3,4,5):
    return "Midnight"
  elif row['Hour'] in (6,7,8,9,10,11,12):
    return "Morning"
  elif row['Hour'] in (13,14,15,16):
    return "Afternoon"
  elif row['Hour'] in (17,18,19):
    return "Evening"
  else:
    return "Night"


def DataandResampling(data, col_date, target, resamplingperiod, label):
    ''' Plot the whole data vs. resampling mean'''
    
    if data[col_date].dtypes == 'datetime64':
        data[col_date] = data[col_date]
    else:
        data[col_date] = pd.to_datetime(data[col_date])

    ## Set the index to date 
    data = data.set_index(col_date) 
        
    ## plot figure    
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.plot(data[target], marker='.', linestyle='-', linewidth=0.5, label=label)
    ax.plot(data[target].resample(resamplingperiod).mean(), marker='o', markersize=6, linestyle='-', label= resamplingperiod + 'ly' + ' Mean Resample')
    ax.xaxis.set_tick_params(rotation=45)
    ax.set_ylabel(target)
    ax.legend(loc = 'upper right')


def YearlyDailyAgg(data, col_date, target, period):
    ''' Aggregate target by year, season, week using date column'''
    
    if data[col_date].dtypes == 'datetime64':
        data[col_date] = data[col_date]
    else:
        data[col_date] = convert_date(data, col_date)
        
    ## plot figure    
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 20))
    fig.subplots_adjust(hspace=0.6, wspace=0.4)

    
    ## Create time features
    date_cols(data, col_date)
    
    data['weekend'] = np.where((data['weekday'] == 5) | (data['weekday'] == 6), 1 ,0) 
    data['season'] = data.apply(create_season, axis = 1)
    

    data.groupby(period)[target].mean().plot.bar(ax = ax1)
    ax1.set_xlabel(period)
    ax1.set_ylabel(target + ' avg')
    ax1.set_title(f'{period} {target} avg')

    ### Mean, median and std of each year
    data.groupby(period).agg({target: ['mean', 'median', 'std']}).plot.bar(ax = ax2)
    ax2.set_title(target +' mean/median/std')
    ax2.set_ylabel(target + ' avg')
    ax2.legend(loc = "best", labels = ['Mean', 'Median', 'Std']);

    ### Consumption by HOUR
    data.groupby(['Hour', period])[target].mean().unstack().plot(style=[':', '--', '-'], ax = ax3)
    ax3.set_title(f'{period} {target} avg by HOUR')
    ax3.set_ylabel(target + ' avg')
    ax3.set_xlim((0,25))

        
def MonthlyWeeklyAgg(data, col_date, target, period):
    ''' Aggregate by month, week or day'''
    
    if data[col_date].dtypes == 'datetime64':
        data[col_date] = data[col_date]
    else:
        data[col_date] = convert_date(data, col_date)
        
    ## plot figure    
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 20))
    fig.subplots_adjust(hspace=0.6, wspace=0.4)

            
    ## Create time features
    date_cols(data, col_date)


    order_month = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']

    order_week = ['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']
    
    
    if period == 'Month':
        data["Month_name"] = data[period].map({1:"JAN", 2:"FEB",3:"MAR",4:"APR",5:"MAY",6:"JUN",
                                            7:"JUL",8:"AUG",9:"SEP",10:"OCT",11:"NOV",12:"DEC"
                                               })
        data.groupby('Month_name')[target].mean().reindex(index = order_month).plot.bar(ax = ax1)
        
        ### Mean, median and std of each year
        data.groupby('Month_name').agg({target: ['mean', 'median', 'std']}).reindex(index = order_month).plot(style=[':', '--', '-'],
                                                                                                            ax = ax2)
        
        ### Period consumption by HOUR
        df_hour = data.groupby(['Hour', 'Month_name'])[target].mean().unstack()
        df_hour = df_hour[order_month]
        df_hour.columns.name = None
        sns.heatmap(df_hour, cmap="YlGnBu", ax = ax3)
        
    elif period == 'weekday':
        data['weekday_name'] = data[period].map({0:'Mon', 1:'Tues', 2:'Wed', 3:'Thurs', 4:'Fri', 5:'Sat', 6:'Sun'})
        data.groupby('weekday_name')[target].mean().reindex(index = order_week).plot.bar(ax = ax1)

        ### Mean, median and std of each week
        data.groupby('weekday_name').agg({target: ['mean', 'median', 'std']}).reindex(index = order_week).plot(style=[':', '--', '-'],
                                                                                                            ax = ax2)
        
        ### Period EA by HOUR
        data.groupby(['Hour', 'weekday_name'])[target].mean().unstack().plot(style=[':', '--', '-'], ax = ax3)
        ax3.set_title(f'{period} {target} avg')
        ax3.set_ylabel(target + ' avg')
        
    else:
        data.groupby(period)[target].mean().plot(ax = ax1)

        ### Mean, median and std of each year
        data.groupby(period).agg({target: ['mean', 'median', 'std']}).plot(style=[':', '--', '-'], ax = ax2)

    
        ### Period consumption by HOUR
        df_hour = data.groupby(['Hour', period])[target].mean().unstack()
        df_hour.columns.name = None
        sns.heatmap(df_hour, cmap="YlGnBu", ax = ax3)
        
    ax1.set_xlabel(period)
    ax1.set_ylabel(target + ' avg')
    ax1.set_title(f'{period} {target} avg') 
    
    ax2.set_title(target + ' mean/median/std')
    ax2.set_xlabel('')
    ax2.set_ylabel(target + ' avg')
    ax2.legend(loc = "best", labels = ['Mean', 'Median', 'Std'])


def TSAggregationEda(data, col_date, target, period):
    ''' Create time series EDA'''
   
    if period in ('Year', 'season', 'weekend'):
        YearlyDailyAgg(data, col_date, target, period)
    else:  
        MonthlyWeeklyAgg(data, col_date, target, period)

def heatmapGroupby(data, col_date, target, row , col, func):

    if data[col_date].dtypes == 'datetime64':
        data[col_date] = data[col_date]
    else:
        data[col_date] = convert_date(data, col_date)
    
    ## Create time features
    order_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    order_month = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
    date_cols(data, col_date)
    data["Month_name"] = data[col].map({1:"JAN", 2:"FEB",3:"MAR",4:"APR",5:"MAY",6:"JUN",
                                            7:"JUL",8:"AUG",9:"SEP",10:"OCT",11:"NOV",12:"DEC"})
    df_g = data.groupby([row, col])[target].agg(func).unstack()
    if col == 'weekday_name':
      df_g = df_g[order_week]
    else:
        df_g = df_g[order_month]
    df_g.columns.name = None
    plt.figure(figsize=(10, 8))
    sns.heatmap(df_g, cmap="YlGnBu")

def missing_values_table(df):
  mis_val = df.isnull().sum()
  mis_val_percent = 100 * df.isnull().sum() / len(df)
  mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
  mis_val_table_ren_columns = mis_val_table.rename(
  columns = {0 : 'Missing Values', 1 : '% of Total Values'})
  mis_val_table_ren_columns = mis_val_table_ren_columns[
      mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)
  print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n" "There are " + str(mis_val_table_ren_columns.shape[0]) +
        " columns that have missing values.")
  return mis_val_table_ren_columns


In [10]:
info_data(df, info = True, head = False, shape = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7979 entries, 0 to 7978
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              7979 non-null   object 
 1   timestamp         7979 non-null   int64  
 2   consumption       7979 non-null   float64
 3   solar             7979 non-null   float64
 4   alwaysOn          7979 non-null   float64
 5   gridImport        7979 non-null   float64
 6   gridExport        7979 non-null   float64
 7   selfConsumption   7979 non-null   float64
 8   selfSufficiency   7979 non-null   float64
 9   active            7979 non-null   object 
 10  reactive          7979 non-null   object 
 11  voltages          7979 non-null   object 
 12  phaseVoltages     7979 non-null   object 
 13  currentHarmonics  7979 non-null   object 
 14  voltageHarmonics  7979 non-null   object 
dtypes: float64(7), int64(1), object(7)
memory usage: 935.2+ KB
------------------------------

In [13]:
df.date.nunique()

7979

In [14]:
df.nunique()

date                7979
timestamp           7979
consumption         7963
solar               7876
alwaysOn             714
gridImport          7946
gridExport             1
selfConsumption        2
selfSufficiency     3092
active              7979
reactive            7979
voltages             280
phaseVoltages        280
currentHarmonics       1
voltageHarmonics       1
dtype: int64

In [18]:
#checking null values:
missing_values_table(df)

Your selected dataframe has 15 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [19]:
describe(df)

,timestamp,consumption,solar,alwaysOn,gridImport,gridExport,selfConsumption,selfSufficiency
count,7.979000e+03,7979.000000,7979.000000,7979.000000,7979.000000,7979.0,7979.000000,7979.000000
mean,1.598709e+12,2972.051785,876.549230,1388.742183,2095.501074,0.0,99.197894,28.723341
std,8.313874e+09,1377.897376,910.770464,455.360344,906.496027,0.0,8.920603,11.511068
min,1.584288e+12,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
25%,1.591544e+12,1904.085500,573.783500,1269.608000,1270.095500,0.0,100.000000,19.735000
median,1.598724e+12,2400.243000,697.616000,1532.403000,1637.155000,0.0,100.000000,28.800000
75%,1.605904e+12,3745.053500,808.778500,1678.789000,2899.423000,0.0,100.000000,35.310000
max,1.613084e+12,10705.265000,8516.431000,3825.575000,3986.403000,0.0,100.000000,92.240000


In [20]:
## convert date to datetime and create date attribute
df = date_cols(df, 'date')

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7979 entries, 0 to 7978
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              7979 non-null   object 
 1   timestamp         7979 non-null   int64  
 2   consumption       7979 non-null   float64
 3   solar             7979 non-null   float64
 4   alwaysOn          7979 non-null   float64
 5   gridImport        7979 non-null   float64
 6   gridExport        7979 non-null   float64
 7   selfConsumption   7979 non-null   float64
 8   selfSufficiency   7979 non-null   float64
 9   active            7979 non-null   object 
 10  reactive          7979 non-null   object 
 11  voltages          7979 non-null   object 
 12  phaseVoltages     7979 non-null   object 
 13  currentHarmonics  7979 non-null   object 
 14  voltageHarmonics  7979 non-null   object 
 15  Year              7979 non-null   int64  
 16  Month             7979 non-null   int64  
